<a href="https://colab.research.google.com/github/B21-CAP0133/verify-android-app/blob/master/scrap/02_VERIFY_Search_from_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [ ]:
def read_db():
  df = pd.read_csv('dB_tbh.csv')

  judul = db['Judul'].apply(str).values.tolist()
  isi = db['Isi berita bersih'].apply(str).values.tolist()
  url = db['Link'].values.tolist()
  date = db['Tanggal'].values.tolist()

  return judul, isi, url, date

In [ ]:
def get_cosine_sim(*strs): 
    vectors = [t for t in get_vectors(*strs)]
    return cosine_similarity(vectors)
    
def get_vectors(*strs):
    text = [t for t in strs]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [ ]:
def cek_kesamaan(headline_input, pembanding):
  cosine_sim = []
  for p in pembanding:
    cs = get_cosine_sim(headline_input, p).min()
    cosine_sim.append(cs)

  maks = cosine_sim.index(max(cosine_sim))
  top = pembanding[maks]

  return max(cosine_sim), top

In [ ]:
def write_json(judul, url, date, isi):
  to_json = {
      'code': 200,
      'judul': judul,
      'url': url,
      'tanggal': date,
      'isi': isi
  }
  with open("message.json", "w") as f:
      json.dump(to_json, f)

def write_json_notfound():
  to_json = {
      'code': 404,
      'isi': 'Tidak ditemukan berita'
  }
  with open("message.json", "w") as f:
      json.dump(to_json, f)

In [ ]:
def search_from_db(headline_input):
  judul, isi, url, date = read_db()

  max_cs_j, top_j = cek_kesamaan(headline_input, judul)
  
  # Treshold apakah mencari kesamaan dalam isi berita diperlukan?
  # kondisi ketika pencarian tidak dilakukan
  if max_cs_j > 0:
    idx = judul.index(top_j)
    write_json(top_j, url[idx], date[idx], isi[idx])
    print('berita tersedia')
  
  # kondisi ketika pencarian perlu dilakukan
  else:
    max_cs_i, top_i = cek_kesamaan(headline_input, isi)

    # kondisi ketika tidak ditemukan kecocokan sama sekali
    if max_cs_i == 0:
      write_json_notfound()
      print('berita belum tersedia')
    else:
      idx = judul.index(top_i)
      write_json(top_i, url[idx], date[idx], isi[idx])
      print('berita tersedia')